In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import numpy as np
import seaborn as sns

In [4]:
referrals_big = pd.read_csv('data/referrals_big.csv')

In [6]:
pd.set_option('display.max_columns', 100)

In [14]:
referrals_big = referrals_big[referrals_big['entity_from'] == 1.0]

In [17]:
referrals_big.to_csv('data/referrals_big_filtered')

In [19]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

communities = pd.read_csv('data/communities')

666

In [21]:
to_communities = communities[communities['from_npi'].isna()].drop(columns = ['from_npi']).rename(columns={'communityId':'to_communityid'})

In [22]:
from_communities = communities[communities['to_npi'].isna()].drop(columns = ['to_npi']).rename(columns={'communityId':'from_communityid'})

In [23]:
to_communities['to_npi']=to_communities['to_npi'].astype(int)
from_communities['from_npi']=from_communities['from_npi'].astype(int)

In [26]:
to_communities

,to_npi,to_communityid
492,1013012616,23818
493,1043218654,24032
494,1902804271,24032
495,1174500714,9895
496,1164486916,541
...,...,...
40822,1508053877,40822
40823,1598098600,13667
40824,1689099939,13667
40825,1669872735,20824


In [27]:
tax_hop_comm = pd.merge(referrals_big,to_communities, on = 'to_npi')

In [29]:
tax_hop_comm = pd.merge(tax_hop_comm, from_communities, on = 'from_npi')

In [35]:
tax_hop_comm['organization_to'] = tax_hop_comm['organization_to'].fillna('none')

In [39]:
tax_hop_comm[tax_hop_comm['organization_to'].str.contains('VANDERBILT')]['to_communityid'].unique()

array([40016, 19175], dtype=int64)

In [40]:
vanderbilt_communities = [40016, 19175]

In [44]:
tax_hop_comm[~tax_hop_comm['from_communityid'].isin(vanderbilt_communities)]['from_npi'].nunique()

19754

In [43]:
tax_hop_comm[tax_hop_comm['from_communityid'].isin(vanderbilt_communities)]['from_npi'].nunique()

3562

19754 doctors outside of vandy communities, 3562 in vandy community

In [82]:
pd.DataFrame(tax_hop_comm[~tax_hop_comm['from_communityid'].isin(vanderbilt_communities)]
             .groupby(['city_from','state_from'])
             ['patient_count']
             .sum()).reset_index().sort_values('patient_count', ascending = False).head(10).city_from.to_list()

['NASHVILLE',
 'BRENTWOOD',
 'MURFREESBORO',
 'COLUMBIA',
 'HENDERSONVILLE',
 'HERMITAGE',
 'CLARKSVILLE',
 'GALLATIN',
 'CHATTANOOGA',
 'DICKSON']

In [86]:
pd.DataFrame(tax_hop_comm[tax_hop_comm['city_from'].isin(['NASHVILLE',
 'BRENTWOOD',
 'MURFREESBORO',
 'COLUMBIA',
 'HENDERSONVILLE',
 'HERMITAGE',
 'CLARKSVILLE',
 'GALLATIN',
 'CHATTANOOGA',
 'DICKSON'])].groupby(['organization_to','classification_to'])['patient_count'].sum()).sort_values('patient_count', ascending = False).reset_index()

,organization_to,classification_to,patient_count
0,none,Internal Medicine,1106724
1,none,Radiology,958093
2,VANDERBILT UNIVERSITY MEDICAL CENTER,Internal Medicine,534903
3,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,416595
4,RADIOLOGY ALLIANCE PC,Radiology,405177
...,...,...,...
1096,"SPRINGFIELD RESIDENTIAL, LLC",Assisted Living Facility,11
1097,"SMYRNA RESIDENTIAL, LLC",Assisted Living Facility,11
1098,"SELECT PHYSICAL THERAPY HOLDINGS, INC.",Clinic/Center,11
1099,"RACHEL B. DORRIS, LCSW, LLC",Social Worker,11


In [47]:
tax_hop_comm[tax_hop_comm['from_communityid'].isin(vanderbilt_communities)].value_counts('state_from')

state_from
TN    31568
KY      266
OH      216
NC      194
SC      167
FL      152
AL      139
TX      120
MO      120
GA       91
WI       72
MA       65
CO       64
IN       54
MI       47
VA       46
NY       41
CT       32
LA       29
WA       28
CA       27
MN       27
MS       23
PA       20
MD       18
AZ       16
AR       16
DC       13
ME        9
NV        7
IL        7
WV        6
UT        5
KS        5
OK        4
RI        4
NJ        4
NE        4
MT        4
ID        4
NM        2
AK        1
dtype: int64

In [51]:
pd.DataFrame(tax_hop_comm.groupby('from_communityid')['patient_count'].sum()).sort_values('patient_count', ascending = False).head(10).index

Int64Index([555, 19175, 9895, 40016, 23818, 20824, 25755, 24032, 33601, 20838], dtype='int64', name='from_communityid')

In [61]:
tax_hop_comm[(tax_hop_comm['to_communityid']
             .isin([555, 19175, 9895, 40016, 23818, 20824, 25755, 24032, 33601, 20838])) & 
             (tax_hop_comm['classification_to'] == 'General Acute Care Hospital')].value_counts('organization_to')

organization_to
VANDERBILT UNIVERSITY MEDICAL CENTER       2771
HCA HEALTH SERVICES OF TENNESSEE, INC.     1727
SAINT THOMAS WEST HOSPITAL                 1193
MAURY REGIONAL HOSPITAL                     698
SAINT THOMAS RUTHERFORD HOSPITAL            424
WILLIAMSON COUNTY HOSPITAL DISTRICT         372
HTI MEMORIAL HOSPITAL CORPORATION           332
HENDERSONVILLE HOSPITAL CORPORATION         312
SUMNER REGIONAL MEDICAL CENTER LLC          261
CENTRAL TENNESSEE HOSPITAL CORPORATION      219
NORTHCREST MEDICAL CENTER                   168
SAINT THOMAS RIVER PARK HOSPITAL, LLC       164
SHELBYVILLE HOSPITAL COMPANY LLC             96
MACON COUNTY GENERAL HOSPITAL, INC.          75
RIVERVIEW MEDICAL CENTER LLC                 65
SUMNER REGIONAL HEALTH SYSTEMS, INC          46
SAINT THOMAS DEKALB HOSPITAL, LLC            27
TROUSDALE MEDICAL CENTER LLC                 23
SAINT THOMAS STONES RIVER HOSPITAL, LLC      15
SETON CORPORATION                             7
SRHS HOLDINGS LLC       

Even in the top communities outside of the vanderbilt community, the top referrals go to Vanderbilt, followed by HCS, st. Thomas, and maury regional

In [70]:
referrals_big['from_npi'].nunique()

23316

0

In [67]:
referrals_big

,from_npi,referral_id,entity_from,name_from,organization_from,taxonomy_code_from,classification_from,specialization_from,address_line1_from,address_line2_from,city_from,state_from,zip_from,to_npi,entity_to,name_to,organization_to,taxonomy_code_to,classification_to,specialization_to,address_line1_to,address_line2_to,city_to,state_to,zip_to,patient_count,transaction_count,average_day_wait,std_day_wait,address_from,address_to
0,1821080961,18854880,1.0,"SHERWOOD, WILLIAM",NaN,207Q00000X,Family Medicine,NaN,302 N CONGRESS BLVD,NaN,SMITHVILLE,TN,37166,1013012616,2.0,NaN,"CRIPPS, HOOPER & RHODY, PLLC",207Q00000X,Family Medicine,NaN,400 E PUBLIC SQUARE,NaN,ALEXANDRIA,TN,37012,91,238,9.924,40.266,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012"
4,1841282779,131966153,1.0,"RHODY, KEVIN",NaN,207Q00000X,Family Medicine,NaN,302 N CONGRESS BLVD,NaN,SMITHVILLE,TN,37166,1013012616,2.0,NaN,"CRIPPS, HOOPER & RHODY, PLLC",207Q00000X,Family Medicine,NaN,400 E PUBLIC SQUARE,NaN,ALEXANDRIA,TN,37012,94,236,7.695,36.447,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012"
5,1770575607,146169993,1.0,"COOPER, STEVEN",NaN,207Q00000X,Family Medicine,NaN,302 N CONGRESS BLVD,NaN,SMITHVILLE,TN,37166,1013012616,2.0,NaN,"CRIPPS, HOOPER & RHODY, PLLC",207Q00000X,Family Medicine,NaN,400 E PUBLIC SQUARE,NaN,ALEXANDRIA,TN,37012,115,320,2.725,18.756,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012"
8,1538109103,4666981,1.0,"NAU, PAUL",NaN,2085R0202X,Radiology,Diagnostic Radiology,3024 BUSINESS PARK CIR,NaN,GOODLETTSVILLE,TN,37072,1043218654,1.0,"MILLER, TIMOTHY",NaN,208VP0014X,Pain Medicine,Interventional Pain Medicine,5801 CROSSINGS BLVD,NaN,ANTIOCH,TN,37013,116,116,33.750,56.887,"3024 BUSINESS PARK CIR, GOODLETTSVILLE, TN 37072","5801 CROSSINGS BLVD, ANTIOCH, TN 37013"
14,1457306565,46830942,1.0,"WUNDER, DANIEL",NaN,2085R0202X,Radiology,Diagnostic Radiology,3024 BUSINESS PARK CIR,NaN,GOODLETTSVILLE,TN,37072,1043218654,1.0,"MILLER, TIMOTHY",NaN,208VP0014X,Pain Medicine,Interventional Pain Medicine,5801 CROSSINGS BLVD,NaN,ANTIOCH,TN,37013,51,51,38.588,57.564,"3024 BUSINESS PARK CIR, GOODLETTSVILLE, TN 37072","5801 CROSSINGS BLVD, ANTIOCH, TN 37013"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240548,1558355941,30083148,1.0,"JOSEPH, JOJU",NaN,207RN0300X,Internal Medicine,Nephrology,270 EAST MAIN STREET,SUITE 200,GALLATIN,TN,37066,1669872735,2.0,NaN,"RIVERVIEW PHYSICIAN PRACTICES, LLC",363LF0000X,Nurse Practitioner,Family,14 MAIN STREET,SUITE B,GORDONSVILLE,TN,38563,31,89,23.101,31.695,"270 EAST MAIN STREET, SUITE 200, GALLATIN, TN ...","14 MAIN STREET, SUITE B, GORDONSVILLE, TN 38563"
240554,1023099074,114150220,1.0,"LAMBERT, HEATHER",NaN,363LW0102X,Nurse Practitioner,Women's Health,507 GORDONSVILLE HWY,SUITE 203,GORDONSVILLE,TN,38563,1669872735,2.0,NaN,"RIVERVIEW PHYSICIAN PRACTICES, LLC",363LF0000X,Nurse Practitioner,Family,14 MAIN STREET,SUITE B,GORDONSVILLE,TN,38563,256,1309,0.214,5.207,"507 GORDONSVILLE HWY, SUITE 203, GORDONSVILLE,...","14 MAIN STREET, SUITE B, GORDONSVILLE, TN 38563"
240561,1184619124,178563129,1.0,"RUTHERFORD, RICHARD",NaN,207Q00000X,Family Medicine,NaN,133 HOSPITAL DR,SUITE 500,CARTHAGE,TN,37030,1669872735,2.0,NaN,"RIVERVIEW PHYSICIAN PRACTICES, LLC",363LF0000X,Nurse Practitioner,Family,14 MAIN STREET,SUITE B,GORDONSVILLE,TN,38563,40,90,23.089,36.995,"133 HOSPITAL DR, SUITE 500, CARTHAGE, TN 37030","14 MAIN STREET, SUITE B, GORDONSVILLE, TN 38563"
240562,1205931565,184096138,1.0,"NOBLE, MICHAEL",NaN,207Q00000X,Family Medicine,NaN,37 PALMER ST,NaN,CALAIS,ME,4619,1669872735,2.0,NaN,"RIVERVIEW PHYSICIAN PRACTICES, LLC",363LF0000X,Nurse Practitioner,Family,14 MAIN STREET,SUITE B,GORDONSVILLE,TN,38563,35,54,1.315,6.815,"37 PALMER ST, CALAIS, ME 04619","14 MAIN STREET, SUITE B, GORDONSVILLE, TN 38563"


In [96]:
hospital_comms = tax_hop_comm[tax_hop_comm['classification_to'].str.contains('Hospital')]

In [102]:
pd.DataFrame(hospital_comms[~hospital_comms['from_communityid'].isin(vanderbilt_communities)]
             .groupby('organization_to')
             ['patient_count'].sum()).sort_values('patient_count', ascending = False).head(10)

,patient_count
organization_to,
"HCA HEALTH SERVICES OF TENNESSEE, INC.",335654
MAURY REGIONAL HOSPITAL,167041
SAINT THOMAS RUTHERFORD HOSPITAL,86567
none,80455
HTI MEMORIAL HOSPITAL CORPORATION,79105
SUMNER REGIONAL MEDICAL CENTER LLC,58834
VANDERBILT UNIVERSITY MEDICAL CENTER,54582
HENDERSONVILLE HOSPITAL CORPORATION,53745
CENTRAL TENNESSEE HOSPITAL CORPORATION,42194


Largest hospital outside of the vanderbilt community is HCA. Explore the doctors outside of vanderbilt community that still refer to vanderbilt. Could be an interesting insight because they already refer to vanderbilt hospitals. Also look at specializations

In [104]:
conditions = [hospital_comms['organization_to'].str.contains('VANDERBILT'), ~hospital_comms['organization_to'].str.contains('VANDERBILT')]
choices = ['Vanderbilt','Other']

hospital_comms['Vanderbilt?'] = np.select(conditions, choices)

<ipython-input-104-a1528850b05c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_comms['Vanderbilt?'] = np.select(conditions, choices)


In [ ]:
This is creating dataframe of npis out of the vanderbilt communities that may still refer to vanderbilt

In [107]:
outside_vandy = pd.DataFrame(hospital_comms[~hospital_comms['from_communityid'].isin(vanderbilt_communities)]
             .groupby(['from_npi','Vanderbilt?'])
             ['patient_count'].sum()).reset_index()

In [111]:
outside_vandy_docs = outside_vandy[outside_vandy['Vanderbilt?'] == 'Vanderbilt'].from_npi.to_list()

In [114]:
recs = outside_vandy[outside_vandy['from_npi'].isin(outside_vandy_docs)].pivot(index = 'from_npi',columns = 'Vanderbilt?', values = 'patient_count')

In [116]:
recs['difference'] = abs(recs['Other'] - recs['Vanderbilt'])

In [121]:
recs = recs.sort_values('difference').dropna()

In [122]:
recs

Vanderbilt?,Other,Vanderbilt,difference
from_npi,,,
1790757755,46.0,45.0,1.0
1720452360,48.0,47.0,1.0
1528046109,22.0,23.0,1.0
1114956687,163.0,164.0,1.0
1972936532,45.0,43.0,2.0
...,...,...,...
1073856696,2155.0,68.0,2087.0
1932465812,2435.0,118.0,2317.0
1942284658,3155.0,70.0,3085.0


In [123]:
recs.to_csv('data/recs.csv')